In [11]:
import os
import sys
sys.path.append('..')
from data_center.tej_handler import TEJHandler
from data_center.tw_monitor import TWMarketMonitor

from pymongo import MongoClient
import pandas as pd
import tejapi

In [12]:
TEJHandler().get_tej_data_table()

,0
融資券資料,TWN/EWGIN
單季財務資料表,TWN/EWIFINQ
指數日行情,TWN/EWIPRCD
指數屬性表,TWN/EWIPRCSTD
證券屬性表,TWN/EWNPRCSTD
證券交易資料表,TWN/EWPRCD
報酬率資訊表,TWN/EWPRCD2
月營收資料表,TWN/EWSALE
三大法人資料,TWN/EWTINST1


In [9]:
source_code = "TWN/EWPRCD2"
rawdata_df = tejapi.get(source_code, chinese_column_name=True,mdate={'gt':'2024-01-01'}) 
pdata_df = TEJHandler().pdata_pipline(rawdata_df, source_code)
pmart = TEJHandler().pmart_pipline(pdata_df, source_code)
pmart

Unsupported data code. Please use one of the following data codes with the correct column names to rename:
'TWN/EWGIN'
'TWN/EWIFINQ'
'TWN/EWIPRCD'
'TWN/EWIPRCSTD'
'TWN/EWNPRCSTD'
'TWN/EWPRCD'
'TWN/EWSALE'
'TWN/EWTINST1'


/Users/tedting/anaconda3/envs/quant311/lib/python3.11/site-packages/tejapi/get.py:45: UserWarning: 如需要更多資料，請設定參數 paginate=True，詳細訊息: 
  warnings.warn(Message.WARN_PAGE_LIMIT_EXCEEDED, UserWarning)


{}

In [8]:
keys_list = list(pmart.keys())
keys_list

['開盤價',
 '最高價',
 '最低價',
 '收盤價',
 '成交量(千股)',
 '交易所本益比',
 '流通股數(千股)',
 '交易所淨值市價比',
 '現金股利率',
 '開盤價-除權息',
 '最高價-除權息',
 '最低價-除權息',
 '收盤價-除權息']

In [10]:
see = pd.read_pickle('/Users/tedting/Documents/old_financial_report.pkl')
see 

,證券碼,財務資料日,季別,合併(Y/N),單季(Q)/單半年(H),月份,幣別,財報發布日,每股盈餘,ROA(C) 稅前息前折舊前,...,營業利益,營業外收入及支出,所得稅費用,合併總損益,歸屬母公司淨利（損）,折舊－CFO,攤提－CFO,來自營運之現金流量,投資活動之現金流量,籌資活動之現金流量
0,1101,200506.0,2,Y,H,6,NTD,2006-08-31,1.25,6.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1101,200512.0,4,Y,H,12,NTD,2007-04-24,0.72,4.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1101,200606.0,2,Y,H,6,NTD,2006-08-31,0.81,4.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1101,200612.0,4,Y,H,12,NTD,2007-04-24,1.42,6.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1101,200706.0,2,Y,H,6,NTD,2007-08-29,1.17,5.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106048,0,NaN,0,nan,nan,0,nan,NaT,NaN,NaN,...,-7798.0,4466.0,374.0,-3706.0,-137.0,5854.0,-41.0,-23620.0,-1166.0,-5212.0
106049,0,NaN,0,nan,nan,0,nan,NaT,NaN,NaN,...,-2693.0,-12691.0,5079.0,-20463.0,-2617.0,6200.0,200.0,60761.0,22922.0,1942.0
106050,0,NaN,0,nan,nan,0,nan,NaT,NaN,NaN,...,-16552.0,3963.0,-73.0,-12516.0,-12620.0,8787.0,379.0,-61344.0,-378.0,-5140.0
106051,0,NaN,0,nan,nan,0,nan,NaT,NaN,NaN,...,-4967.0,743.0,-12.0,-3984.0,1657.0,7993.0,379.0,26027.0,-2347.0,-4259.0
